**This notebook allows to simulate data, classify it using a DLBM model and evaluate the model in a controlled environnement. The model is a dynamic LBM `dLBM` for data represented as a series of adjacency matrices.**

# Imports

In [ ]:
import pathlib
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message='Deprecation')

from dcblockmodels.models.dlbm import dLBM

from dcblockmodels import metrics, plot, data
from dcblockmodels.models.utils import general, init
from dcblockmodels.models.utils.smoothing_schedule import SmoothingSchedule

# Data

## Sampling the data

In [ ]:
# whether we sample from a SBM or LBM
model_type_data = 'LBM'
# in case of SBM, whether the graph is directed
directed = True
# number of time steps
T = 20
# nb of row nodes, nb of column nodes
N, D = 200, 300
# nb row clusters, nb of column clusters 
Kz, Kw = 3, 4

In [ ]:
level_alpha = 'easy'
level_beta = 'easy'
level_pi = 'easy'
level_rho = 'easy'

alphas_dirichlet = {
    'very_easy': 10,
    'easy': 8,
    'medium': 6,
    'hard': 4
}
diag_vals = {
  'diag': 0,
  'easy': .9,
  'medium': .75,
  'hard': .6
}

alpha = data.generate_initial_proportions(Kz, alphas_dirichlet[level_alpha])
beta = data.generate_initial_proportions(Kw, alphas_dirichlet[level_beta])
prior_init = {'alpha': alpha, 'beta': beta}

pi = data.generate_diag_transition_matrix(Kz, diag_vals[level_pi]) 
rho = data.generate_diag_transition_matrix(Kw, diag_vals[level_rho])
prior_trans = {'pi': pi, 'rho': rho}

alpha, pi, beta, rho

In [ ]:
with_margins = True # True, False
constant_margins = False # True, False
start, stop, step = 1, 50, .1
order_power_law = -1.5 # margins ~ Unif(start, stop)^order_power_law
# mu ~ AR1 : mu_{t+1} = N(a mu_t + c, sigma2) (c s.t. mu increasing if sigma2 = 0)
ar_margins, a_ar, sigma2_ar = True, 1.1, .05 

if with_margins:
    mu, nu = data.generate_margins(
        T, N, D, constant_margins, start, stop, step,
        directed, order_power_law,
        ar_margins, a_ar, sigma2_ar
    )
    margins = {'mu': mu, 'nu': nu}
else:
    margins = None
    
margins

In [ ]:
with_absent_nodes = False # True, False
min_proba_t = .0
max_proba_t = .2
proba_absent = None

if with_absent_nodes:
    absent_row_nodes = data.sample_absent_nodes(
        T, N,
        min_proba_t=min_proba_t,
        max_proba_t=max_proba_t,
        proba_absent=proba_absent
    )
    if not directed:
        absent_col_nodes = absent_row_nodes.copy()
    else:
        absent_col_nodes = data.sample_absent_nodes(
            T, D,
            min_proba_t=min_proba_t,
            max_proba_t=max_proba_t,
            proba_absent=proba_absent
        )
else:
    absent_row_nodes, absent_col_nodes = [], []

absent_nodes = {
    'absent_row_nodes': absent_row_nodes,
    'absent_col_nodes': absent_col_nodes
}
absent_nodes

In [ ]:
# scaling factor for the matrix gamma : determines the separability level
# lower is harder and more sparse
gamma_0 = .05

# defines the sparsity in a block
# block_sparsity_matrix[t, k, l] is the proba of a zero
# in block (k, l) at time t
# corresponds to the \beta_{kl}^t of Matias
block_sparsity_matrix = None
# block_sparsity_matrix = 0.1 * np.ones((Kz, Kw), dtype='float')

# if we add gaussian noise to the sampled graph
# not advised since it can make models with lower
# complete data log likelihood give better classification results
# than model with higher complete data log likelihood
noise_level_ = 0.


if Kz == 3 and Kw == 4:
    gamma  = gamma_0 * np.array([
        [1, 2, 3, 1 ],
        [3, 1, 2, 3 ],
        [2, 3, 1, 4 ]
    ])
elif Kz == 3 and Kw == 3:
    gamma  = gamma_0 * np.array([
        [1, 2, 3],
        [3, 1, 2],
        [2, 3, 1]
    ])
else:
    raise ValueError

if T > 1:
    gamma = np.stack([gamma for _ in range(T)], axis=0)
    if block_sparsity_matrix is not None:
        block_sparsity_matrix = np.stack([block_sparsity_matrix for _ in range(T)], axis=0)
        
gamma

In [ ]:
dimensions = {'N': N, 'D': D}
n_clusters = {'Kz': Kz, 'Kw': Kw}

self_loops = True
dtype = 'int32'

X, Z, W = data.generate_data(
    T,
    model_type_data,
    dimensions,
    n_clusters,
    prior_init,
    prior_trans,
    gamma,
    with_margins,
    margins,
    self_loops,
    directed,
    noise_level_,
    with_absent_nodes,
    absent_nodes,
    dtype,
    block_sparsity_matrix
)

## Plot

### Block view & link with matrix factorization

In [ ]:
t_plot = 0
X_ = X[t_plot]
Z_ = Z[t_plot]
W_ = W[t_plot]
gamma_ = gamma[t_plot]

row_indices = np.argsort(Z_.astype(int))
col_indices = np.argsort(W_.astype(int))

cmap = sns.cubehelix_palette(light=1., as_cmap=True)
f, ax = plt.subplots(1, 4, figsize=(4 * 5, 5))

sns.heatmap(X_, ax=ax[0], cbar=False, square=False, xticklabels=False, yticklabels=False, cmap=cmap)
ax[0].set_title('Raw data')

sns.heatmap(X_[row_indices, :], ax=ax[1], cbar=False, square=False, xticklabels=False, yticklabels=False, cmap=cmap)
ax[1].set_title('Row-reorganized data')

sns.heatmap(X_[np.ix_(row_indices, col_indices)], ax=ax[2], cbar=False, square=False, xticklabels=False, yticklabels=False, cmap=cmap)
ax[2].set_title('Row and column-reorganized data')

Z_encoded = general.encode(Z_, Kz)
W_encoded = general.encode(W_, Kw)
X_approx = Z_encoded.dot(gamma_).dot(W_encoded.T)
sns.heatmap(X_approx[np.ix_(row_indices, col_indices)], ax=ax[3], cbar=False, square=False, xticklabels=False, yticklabels=False, cmap=cmap)
ax[3].set_title('Connectivity-approximized data')

plt.tight_layout()

### Dimensionality reduction with Correspondence Analysis

In [ ]:
t_plot = 0
# t_plot = [t for t in range(T)]
# t_plot = [0, T//2, T - 1]

if type(t_plot) == int:
    t_plot = [t_plot]

n_plots = len(t_plot)
f, ax = plt.subplots(n_plots, 2, figsize=(10, 5 * n_plots))
for i, t in enumerate(t_plot):
    W_plot = W[t] if W is not None else None

    absent_row = [tup[1] for tup in absent_row_nodes if tup[0] == t]
    absent_col = [tup[1] for tup in absent_col_nodes if tup[0] == t]

    plot.CA_plot(
        X[t],
        Z[t], W_plot,
        absent_row, absent_col,
        ax=ax[i]
    )
    ax[i, 0].set_title(f't = {t}')
    ax[i, 1].set_title(f't = {t}')


### True margins

Plot margins over time, in the dynamic case

In [ ]:
n_nodes = 20
f, ax = plt.subplots(1, 2, figsize=(2 * 6, 4))

ax[0].plot(margins['mu'][:, np.random.choice(N, size=n_nodes)]);
ax[0].set_title('True row margins mu');

ax[1].plot(margins['nu'][:, np.random.choice(D, size=n_nodes)]);
ax[1].set_title('True col margins nu');

### Factorial Discriminant Analysis

Measures the level of linear separability of the classes after projection onto R^N using correspondence analysis

See Discriminative Factorial Analysis : http://www.math.u-bordeaux.fr/~mchave100p/wordpress/wp-content/uploads/2013/10/AFD.pdf

In [ ]:
t_plot = 0
t_plot = [t for t in range(T)]
t_plot = [0, T//2, -1]

n_components = 3

f, ax = plt.subplots(len(t_plot), 2, squeeze=False, sharex=True, sharey=True, figsize=(5 * len(t_plot), 8))
xs = np.arange(n_components, dtype='int')

for i, t in enumerate(t_plot):
    res = metrics.AFD_CA_linear_separation(
        X[t], Z[t], W[t],
        n_components=n_components,
        absent_row_nodes=absent_row_nodes,
        absent_col_nodes=absent_col_nodes
    )
    ax[i, 0].bar(xs, res[0])
    ax[i, 1].bar(xs, res[1])
    ax[i, 0].set_xlabel('factorial axis')
    ax[i, 1].set_xlabel('factorial axis')
    ax[i, 0].set_title(f'Rows, T = {t}')
    ax[i, 1].set_title(f'Cols, T = {t}')

plt.suptitle('CA AFD linear separability', y=1);
plt.tight_layout()

### Distribution of the values of the cells of the data matrix

In [ ]:
t_plot = 0
t_plot = [t for t in range(T)]
t_plot = [0, T//2, -1]

bins = 50
val_min = 1
val_max = 100 # int or None

f, ax = plt.subplots(len(t_plot), 1, sharex=True, sharey=True, figsize=(10, 1.5 * len(t_plot)))

for i, t in enumerate(t_plot):
    values = X[t].flatten()
    values = values[values >= val_min]
    if val_max is not None:
        values = values[values < val_max]
    ax[i].hist(values, bins=bins)
    ax[i].set_title(f'time t = {t}')
    
f.suptitle('Histogram of the values of the cells of the data matrix over time');
plt.tight_layout();

In [ ]:
t_plot = 0
t_plot = [t for t in range(T)]
t_plot = [0, T//2, -1]

bins = 50
val_min = 0
val_max = None #int or None

f, ax = plt.subplots(len(t_plot), 1, sharex=True, sharey=True, figsize=(10, 1.5 * len(t_plot)))

for i, t in enumerate(t_plot):
    values = X[t].sum(0).flatten()
    values = values[values >= val_min]
    if val_max is not None:
        values = values[values < val_max]
    ax[i].hist(values, bins=bins)
    ax[i].set_title(f'time t = {t}')
    
f.suptitle('Histogram of the degrees of the nodes over time');
plt.tight_layout();

# DLBM

### Algo params

In [ ]:
smoothing_schedule = SmoothingSchedule('sigmoid', 50, tau0=1e-3, x0=-6., x1=5.)
smoothing_schedule.plot()

In [ ]:
model_type = 'with_margins' #  # 'with_margins', # 'without_margins'
parameter_smoothing = True # True, False
n_iter_supp_smoothing = 10
sparse_X = True # True, False

n_init = 10
em_type = 'CEM' # 'VEM', 'CEM'
max_iter = 500
tol_iter = 1e-6
min_float = 1e-15
min_proba_Z, min_proba_W = .1, .1
min_proba_mixture_proportions = 1e-1  # to avoid empty clusters
min_margin = 1e-10
min_gamma = 1e-10
prior_diagonal_pi, prior_diagonal_rho = 0., 0. #.2, .2
diag_pi_init, diag_rho_init = .7, .7

init_type = 'skmeans' #'given' # 'skmeans', 'kmeans', 'given'
type_init_margins = 'ones' # ones, X.
given_mu, given_nu = None, None
n_init_clustering = 20
node_perturbation_rate = .15
cluster_perturbation_rate = 0.
threshold_absent_nodes = -1

debug_output = pathlib.Path(r'../dcblockmodels/model_debug_output')
dtype = 'float64'
random_state = None
n_jobs = -1
verbose = 1
model_id = 1

# debug_list contains the names of the parameters fo the models
# or of the variational distribution that we wish to monitor
# during the fitting of the model
# This is done by writing the values of the model to disk
# so it takes time and space. Providing an empty list
# is the normal behavior
debug_list = []  # ['log_alpha', 'gamma', 'log_pi', 'Z', 'mu']

### Initialization

In [ ]:
if init_type == 'given':
    # one could get initial partitions
    # using any clustering algo
    given_Z = init._skmeans_init(
        np.concatenate([X[t] for t in range(T)], axis=1),
        Kz, n_init_clustering, random_state=None, n_jobs=-1
    )
    given_W = init._skmeans_init(
        np.concatenate([X[t] for t in range(T)], axis=0).T,
        Kw, n_init_clustering, random_state=None, n_jobs=-1
    )
else:
    given_Z, given_W = None, None

if sparse_X:
    X_ = [general.to_sparse(X[t]) for t in range(T)]
else:
    X_ = X.copy()

### Fitting the model

In [ ]:
model = dLBM(
    model_type=model_type,
    em_type=em_type,
    parameter_smoothing=parameter_smoothing,
    Kz=Kz, Kw=Kw,
    n_init=n_init,
    model_id=model_id,
    max_iter=max_iter,
    type_init_margins=type_init_margins,
    smoothing_schedule=smoothing_schedule.schedule,
    n_iter_supp_smoothing=n_iter_supp_smoothing,
    prior_diagonal_pi=prior_diagonal_pi,
    prior_diagonal_rho=prior_diagonal_rho,
    diag_pi_init=diag_pi_init,
    diag_rho_init=diag_rho_init,
    init_type=init_type,
    n_init_clustering=n_init_clustering,
    node_perturbation_rate=node_perturbation_rate,
    cluster_perturbation_rate=cluster_perturbation_rate,
    threshold_absent_nodes=threshold_absent_nodes,
    min_proba_mixture_proportions=min_proba_mixture_proportions,
    min_gamma=min_gamma,
    min_margin=min_margin,
    min_proba_Z=min_proba_Z,
    min_proba_W=min_proba_W,
    dtype=dtype,
    blockmodel_params=None,
    random_state=random_state,
    tol_iter=tol_iter,
    n_jobs=n_jobs, verbose=verbose,
    debug_list=debug_list,
    debug_output=debug_output
)
model.fit(
    X_, 
    given_Z=given_Z, given_W=given_W,
    given_mu=given_mu, given_nu=given_nu
)

## Load/save model

### Save

In [ ]:
model.save(path='../saved_models', modelname='my_model')

### Load

In [ ]:
model = general.load_model('../saved_models/my_model')
model

# Metrics & visualizations

## Partitions and criterion

In [ ]:
Z_model, W_model = model.best_partition(mode='likelihood', n_first=1)[0]
#Z_model, W_model = model.best_partition(mode='consensus: hbgf', n_first=(model.n_init) // 2)[0]

plot.plot_criterions(
    model,
    thr_decrease=1000,
    i_start=0, i_end=-1,
    legend=True
)

## DLBM

### Metrics

In [ ]:
metrics.print_metrics(
    Z_model, W_model, Z, W,
    absent_nodes=model.absent_nodes,
    print_each_timestep=True
)

### Confusion matrices

In [ ]:
for t in range(T):
    print('t = {}'.format(t), end='\n')
    print(metrics.cmat_clustering(confusion_matrix(
        Z_model[t], Z[t])), end='\n\n')

In [ ]:
for t in range(T):
    print('t = {}'.format(t), end='\n')
    print(metrics.cmat_clustering(confusion_matrix(
        W_model[t], W[t])), end='\n\n')

In [ ]:
plot.plot_alluvial(Z)

In [ ]:
plot.plot_alluvial(Z_model)

## Model parameters

`model.best_parameters` : a list of length the number of initializations of the model. Each element of the list is a tuple in the form `(crit, param_dic)`, where `crit` is the best value of the objective criterion of the model of the given init and `param_dic` contains the parameters of the model that gave this `crit` 

In [ ]:
model.best_parameters[0]

Mapping the indexes of the found clusters to the indexes of the true clusters using the Kuhn Munkres/Hungarian algorithm on the confusion matrix

In [ ]:
Z_model, W_model = model.best_partition(mode='likelihood', n_first=1)[0]

cmat_Z = confusion_matrix(
    metrics.get_flat_present_nodes(Z_model, absent_row_nodes),
    metrics.get_flat_present_nodes(Z, absent_row_nodes)
)
cmat_W = confusion_matrix(
    metrics.get_flat_present_nodes(W_model, absent_col_nodes),
    metrics.get_flat_present_nodes(W, absent_col_nodes)
)
    
indexes_Z = linear_sum_assignment(-cmat_Z)[1]
indexes_W = linear_sum_assignment(-cmat_W)[1]

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 6))

gamma_model = model.best_parameters[0][1]['gamma']
reordered_gamma_model = gamma_model[np.ix_(indexes_Z, indexes_W)]

sns.heatmap(reordered_gamma_model, ax=ax[0],  square=True)
sns.heatmap(gamma[0], ax=ax[1], square=True)

ax[0].set_title('Estimated gamma');
ax[1].set_title('True gamma');

In [ ]:
f, ax = plt.subplots(2, 2, figsize=(12, 6))

pi_model = np.exp(model.best_parameters[0][1]['log_pi'])
reordered_pi_model = pi_model[np.ix_(indexes_Z, indexes_Z)]

sns.heatmap(reordered_pi_model, ax=ax[0, 0],  square=True)
sns.heatmap(pi, ax=ax[0, 1], square=True)

ax[0, 0].set_title('Estimated pi')
ax[0, 1].set_title('True pi')

rho_model = np.exp(model.best_parameters[0][1]['log_rho'])
reordered_rho_model = rho_model[np.ix_(indexes_W, indexes_W)]

sns.heatmap(reordered_rho_model, ax=ax[1, 0],  square=True)
sns.heatmap(rho, ax=ax[1, 1], square=True)

ax[1, 0].set_title('Estimated rho')
ax[1, 1].set_title('True rho');

In [ ]:
n_nodes = 20
f, ax = plt.subplots(2, 2, figsize=(2 * 8, 8))

row_nodes = np.random.choice(N, size=n_nodes)
col_nodes = np.random.choice(D, size=n_nodes)

ax[0, 0].plot(margins['mu'][:, row_nodes])
ax[0, 1].plot(model.best_parameters[0][1]['mu'][:, row_nodes])
ax[0, 0].set_title('True row margins mu')
ax[0, 1].set_title('Estimated row margins mu')

ax[1, 0].plot(margins['nu'][:, col_nodes])
ax[1, 1].plot(model.best_parameters[0][1]['nu'][:, col_nodes])
ax[1, 0].set_title('True row margins nu')
ax[1, 1].set_title('Estimated row margins nu');

# Debug : parameters during inference

Get parameter values during the iterations of the algorithm. The parameters we wish to analyze must be given as strings in given in model.debug_list. The parameters are written in the directory `dcblockmodels/model_debug_output`, which should be emptied from time to time.

In [ ]:
debug_dic = model.get_debug()
debug_dic

In [ ]:
# debug_dic['param'][init][iter] : returns the value of the parameter 
# 'param' that was given in self.debug_list
# for the initialization init
# and for the iteration iter
debug_dic['gamma'][0][10]

## Alpha and beta

In [ ]:
plot.plot_alphas_during_optim(debug_dic['log_alpha'])

## Pi and rho

In [ ]:
plot.plot_pi_rho_during_optim(debug_dic['log_pi'])

## Gamma

In [ ]:
plot.plot_gamma_during_optim(debug_dic['gamma'])

## Mu and nu

In [ ]:
plot.plot_mu_nu_during_optim(debug_dic['mu'], indexes=np.random.choice(N, size=(2)))